In [ ]:
using GuSTO
using AstrobeeRobot

In [ ]:
robot = Astrobee3D()
model = AstrobeeSE3()
env = ISSCorner();
add_obstacles1!(env)

x_inits = []
push!(x_inits, [6.5; 0.; 5.]) # Hallway 3
push!(x_inits, [11.; -2.; 5.]) # Hallway 6
push!(x_inits, [11.2; 0.8; 5.6]) # 8 x y z
push!(x_inits, [11.2; -0.8; 5.6]) # 8 x -y z
push!(x_inits, [11.2; -0.8; 4.1]) # 8 x -y -z
push!(x_inits, [8.1; -0.8; 5.6]) # 8 -x -y z
push!(x_inits, [8.1; -0.8; 4.1]) # 8 -x -y -z
push!(x_inits, [8.1; 0.8; 5.6]) # 8 -x y z
push!(x_inits, [8.1; 0.8; 4.1]) # 8 -x y -z
push!(x_inits, [9.6; -0.8; 4.1]) # 8 xmid -y -z

x_goals = []
push!(x_goals, [11.5; 6.9; 4.2]) # 5 x y z
push!(x_goals, [11.5; 6.9; 5.5]) # 5 x y -z
push!(x_goals, [11.5; 3.1; 5.5]) # 5 x -y z
push!(x_goals, [11.5; 3.1; 4.2]) # 5 x -y -z
push!(x_goals, [10.2; 6.9; 5.5]) # 5 -x y z
push!(x_goals, [10.2; 6.9; 4.2]) # 5 -x y -z
push!(x_goals, [10.2; 3.1; 5.5]) # 5 -x -y z
push!(x_goals, [10.2; 3.1; 4.2]) # 5 -x -y -z
push!(x_goals, [10.2; 5.; 5.5]) # 5 -x ymid z
push!(x_goals, [10.2; 5.; 4.2]) # 5 -x ymid -z

x_init = [x_inits[4]; zeros(3); quat2mrp(sqrt(1/3)*[1.; 0.; 1.; 1.]); zeros(3)]
x_goal = [x_goals[6]; zeros(3); quat2mrp([-0.5; 0.5; -0.5; 0.5]); zeros(3)]

N = 80
tf_guess = 70.

PD = ProblemDefinition(robot, model, env, x_init, x_goal);
TOPgusto = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOPgusto)
solve_SCP!(TOSgusto, TOPgusto, solve_gusto_cvx!, init_traj_straightline, "Gurobi", OutputFlag=0)
# solve_SCP!(TOSgusto, TOPgusto, solve_trajopt_cvx!, init_traj_straightline, "Gurobi")#, OutputFlag=0)

In [ ]:
@show TOSgusto.SCPS.converged
@show TOSgusto.SCPS.iterations
@show TOSgusto.SCPS.iter_elapsed_times
@show TOSgusto.SCPS.total_time
@show TOSgusto.SCPS.accept_solution
@show TOSgusto.SCPS.prob_status
@show TOSgusto.SCPS.convergence_measure
@show maximum(abs.(TOSgusto.traj.U[1:3,:]))
@show verify_collision_free(TOSgusto.traj, TOSgusto.SCPS.SCPP)
refined_traj = interpolate_traj(TOSgusto.traj, TOSgusto.SCPS.SCPP)
@show verify_collision_free(refined_traj, TOSgusto.SCPS.SCPP)
;

In [ ]:
r0 = TOSgusto.traj.X[1:3,10]
# r0 = refined_traj.X[1:3,642]
traj = Trajectory([r0 r0], zeros(6,1), 0.)
n_koz = length(env.keepout_zones)
n_obs = length(env.obstacle_set)
println("***Walls***")
for i = 1:n_koz
    @show max(ncsi_obstacle_avoidance_constraints_convexified(traj, traj, TOSgusto.SCPS.SCPP, 1, i), 0)
end
println("\n***Obstacles***")
for i = n_koz+1:n_koz+n_obs
    @show max(ncsi_obstacle_avoidance_constraints_convexified(traj, traj, TOSgusto.SCPS.SCPP, 1, i), 0)
end

In [ ]:
#### Code for animating Astrobee trajectories
vis = Visualizer()
delete!(vis)

vis[:goal]
for (idx,obs) in enumerate(env.keepout_zones)
    setobject!(vis[:goal][:goal], 
        Object(HyperSphere(Point3f0(x_goal[1:3]), 0.1f0),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))
end

vis[:workspace]
for (idx,ws) in enumerate(env.keepin_zones)
    if idx in (5,8)
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3), depthWrite=false)))
    else
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3))))
    end
end

vis[:obstacle]
for (idx,ws) in enumerate(env.obstacle_set)
    setobject!(vis[:workspace][Symbol(string("ws",idx+length(env.keepin_zones)))],
        Object(ws,MeshBasicMaterial(color=RGBA(0.95,0.26,0.26,0.3))))
end

ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);
speed_factor = 1

Qs = Vector{Vector{Float64}}(0)
for k in 1:speed_factor:N
    q = [quat_inv(mrp2quat(TOSgusto.SCPS.traj.X[7:9,k]));TOSgusto.SCPS.traj.X[1:3,k]]
    push!(Qs,q)
end

trans = Translation(14., -1., 7.)
rot = LinearMap(RotZ(-0.6)) ∘ LinearMap(RotY(-0.2))
settransform!(vis["/Cameras/default"], trans ∘ rot)
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1.9)
setprop!(vis["/Cameras/default/rotated/<object>"], "near", 0.05)

sleep(3)
setanimation!(mvis,1:length(Qs),Qs)

plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)

In [ ]:
#### Code for visualizing Astrobee positions

vis = Visualizer()
delete!(vis)

vis[:goal]
for (idx,obs) in enumerate(env.keepout_zones)
    setobject!(vis[:goal][:goal], 
        Object(HyperSphere(Point3f0(x_goal[1:3]), 0.1f0),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))
end

vis[:workspace]
for (idx,ws) in enumerate(env.keepin_zones)
    if idx in (5,8)
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3), depthWrite=false)))
    else
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3))))
    end
end

vis[:obstacle]
for (idx,ws) in enumerate(env.obstacle_set)
    setobject!(vis[:workspace][Symbol(string("ws",idx+length(env.keepin_zones)))],
        Object(ws,MeshBasicMaterial(color=RGBA(0.95,0.26,0.26,0.3))))
end

ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);
ab2 = Astrobee()
mvis2 = MechanismVisualizer(
    ab2.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);

x_inits = []
push!(x_inits, [6.5; 0.; 5.]) # Hallway 3
push!(x_inits, [11.; -2.; 5.]) # Hallway 6
push!(x_inits, [11.2; 0.8; 5.6]) # 8 x y z
push!(x_inits, [11.2; -0.8; 5.6]) # 8 x -y z
push!(x_inits, [11.2; -0.8; 4.1]) # 8 x -y -z
push!(x_inits, [8.1; -0.8; 5.6]) # 8 -x -y z
push!(x_inits, [8.1; -0.8; 4.1]) # 8 -x -y -z
push!(x_inits, [8.1; 0.8; 5.6]) # 8 -x y z
push!(x_inits, [8.1; 0.8; 4.1]) # 8 -x y -z
push!(x_inits, [9.6; -0.8; 4.1]) # 8 xmid -y -z

x_goals = []
push!(x_goals, [11.5; 6.9; 4.2]) # 5 x y z
push!(x_goals, [11.5; 6.9; 5.5]) # 5 x y -z
push!(x_goals, [11.5; 3.1; 5.5]) # 5 x -y z
push!(x_goals, [11.5; 3.1; 4.2]) # 5 x -y -z
push!(x_goals, [10.2; 6.9; 5.5]) # 5 -x y z
push!(x_goals, [10.2; 6.9; 4.2]) # 5 -x y -z
push!(x_goals, [10.2; 3.1; 5.5]) # 5 -x -y z
push!(x_goals, [10.2; 3.1; 4.2]) # 5 -x -y -z
push!(x_goals, [10.2; 5.; 5.5]) # 5 -x ymid z
push!(x_goals, [10.2; 5.; 4.2]) # 5 -x ymid -z

x_init = x_inits[1]
x_goal = x_goals[1]

settransform!(mvis["world"]["body"], Translation(TOSgusto.traj.X[1:3,10]...))
settransform!(mvis2["world"]["body"], Translation(TOSgusto.traj.X[1:3,5]...))

trans = Translation(14., -1., 7.)
rot = LinearMap(RotZ(-0.6)) ∘ LinearMap(RotY(-0.2))
settransform!(vis["/Cameras/default"], trans ∘ rot)
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1.9)
setprop!(vis["/Cameras/default/rotated/<object>"], "near", 0.05)

sleep(3)

plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)

In [ ]:
#### Code for checking whether Astrobee positions are in collision
TOPgusto = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOPgusto)
SCPP = SCPProblem(TOPgusto)
SCPP.param.alg = SCPParam_GuSTO(SCPP.PD.model);
SCPP.param.obstacle_toggle_distance = SCPP.PD.model.clearance;

x_init = [6.5; 0.; 5.] # Hallway 3 
x_init = [11.; -2.; 5.] # Hallway 6
x_init = [11.2; 0.8; 5.6] # 8 x y z
x_init = [11.2; -0.8; 5.6] # 8 x -y z
x_init = [11.2; -0.8; 4.1] # 8 x -y -z
x_init = [8.1; -0.8; 5.6] # 8 -x -y z
x_init = [8.1; -0.8; 4.1] # 8 -x -y -z
x_init = [8.1; 0.8; 5.6] # 8 -x y z
x_init = [8.1; 0.8; 4.1] # 8 -x y -z
x_init = [9.6; -0.8; 4.1] # 8 xmid -y -z 

x_goal = [11.5; 6.9; 4.2] # 5 x y z
x_goal = [11.5; 6.9; 5.5] # 5 x y -z
x_goal = [11.5; 3.1; 5.5] # 5 x -y z
x_goal = [11.5; 3.1; 4.2] # 5 x -y -z
x_goal = [10.2; 6.9; 5.5] # 5 -x y z
x_goal = [10.2; 6.9; 4.2] # 5 -x y -z
x_goal = [10.2; 3.1; 5.5] # 5 -x -y z
x_goal = [10.2; 3.1; 4.2] # 5 -x -y -z

r0 = [6.5; 0.; 5.]
r0 = [10.2; 7.; 5.5]
r0 = TOSgusto.traj.X[1:3,3]
r0 = x_goal
traj = Trajectory([r0 r0], zeros(6,1), 0.)

n_koz = length(env.keepout_zones)
n_obs = length(env.obstacle_set)
println("***Walls***")
for i = 1:n_koz
    @show max(ncsi_obstacle_avoidance_constraints_convexified(traj, traj, SCPP, 1, i), 0)
end
println("\n***Obstacles***")
for i = n_koz+1:n_koz+n_obs
    @show max(ncsi_obstacle_avoidance_constraints_convexified(traj, traj, SCPP, 1, i), 0)
end

In [ ]:
TOPmao = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSmao = TrajectoryOptimizationSolution(TOPmao)
solve_SCP!(TOSmao, TOPmao, solve_mao_cvx!, init_traj_straightline, "Gurobi")

In [ ]:
TOPtrajopt = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOStrajopt = TrajectoryOptimizationSolution(TOPtrajopt)
solve_SCP!(TOStrajopt, TOPtrajopt, solve_trajopt_cvx!, init_traj_straightline, "Gurobi")

In [ ]:
@show TOStrajopt.SCPS.converged
@show TOStrajopt.SCPS.iterations
@show TOStrajopt.SCPS.iter_elapsed_times
@show TOStrajopt.SCPS.total_time
@show TOStrajopt.SCPS.prob_status
@show TOStrajopt.SCPS.convergence_measure